In [132]:
import pandas as pd
import numpy as np
data = pd.read_csv('test_data.csv', header=None)
array = data.values
array

array([[11111],
       [19991],
       [19191],
       [19991],
       [11111]])

In [10]:

numbers = [str(no[0]) for no in data.values.tolist()]

In [76]:
import cv2
import pandas as pd
import numpy as np

data = pd.read_csv('test_data.csv', header=None)
numbers = [str(no[0]) for no in data.values.tolist()]
original_array = []
for idx in range(len(numbers)):
    original_array.append([int(i) for i in numbers[idx]])
original_array = np.array(original_array)
original_array = cv2.copyMakeBorder(original_array, 1,1,1,1,cv2.BORDER_CONSTANT, value=[11])
original_array = np.array(original_array) + 1
original_array

array([[12, 12, 12, 12, 12, 12, 12],
       [12,  2,  2,  2,  2,  2, 12],
       [12,  2, 10, 10, 10,  2, 12],
       [12,  2, 10,  2, 10,  2, 12],
       [12,  2, 10, 10, 10,  2, 12],
       [12,  2,  2,  2,  2,  2, 12],
       [12, 12, 12, 12, 12, 12, 12]], dtype=int32)

In [68]:
nploc = np.where(original_array == 10)
# set 10's to 0
original_array = np.where(original_array == 10, 0, original_array)
nploc
print(original_array)

[[12 12 12 12 12 12 12]
 [12  2  2  2  2  2 12]
 [12  2  0  0  0  2 12]
 [12  2  0  2  0  2 12]
 [12  2  0  0  0  2 12]
 [12  2  2  2  2  2 12]
 [12 12 12 12 12 12 12]]


In [47]:
# Get roi where val = 10, add 1 to neighbouring vals
flashed = (np.array([],dtype='int32'),np.array([],dtype='int32'))
for y,x in zip(nploc[0], nploc[1]):
    roi = original_array[y-1:y+2, x-1:x+2]
    roi_plus1 = np.where(roi != 0, roi + 1, roi)
    original_array[y-1:y+2, x-1:x+2] = roi_plus1

    # Check for 10's at each flash stage if it caused any cascading effect
    check_10 = np.where(original_array == 10)
    # Set found 10's to 0
    original_array = np.where(original_array == 10, 0, original_array)
    # Store loc in tuple
    flashed = (np.append(flashed[0],check_10[0]),np.append(flashed[1],check_10[1]))

print(flashed)
nploc = np.where(original_array == 10)

#every loop set 10's to 0 and keep record

(array([3]), array([3]))


In [47]:
def oct_flash(original_array:np.array, input_loc = []) -> tuple:

    if input_loc:
        coord = input_loc
    else:
        # Find 10's
        coord = np.where(original_array == 10)

    flash_count = len(coord[0])
    # set 10's to 0
    original_array = np.where(original_array == 10, 0, original_array)
    cascade_flashed = (np.array([],dtype='int32'),np.array([],dtype='int32'))

    for y,x in zip(coord[0], coord[1]):

        # Find roi and increment neighbours by 1 if not equal to 0
        roi = original_array[y-1:y+2, x-1:x+2]
        roi_plus1 = np.where(roi != 0, roi + 1, roi)
        original_array[y-1:y+2, x-1:x+2] = roi_plus1

        # Check for 10's at each flash stage if it caused any cascading effect
        check_10 = np.where(original_array == 10)
        # Set found 10's to 0
        original_array = np.where(original_array == 10, 0, original_array)
        # Store loc in tuple
        cascade_flashed = (np.append(cascade_flashed[0],check_10[0]),np.append(cascade_flashed[1],check_10[1]))

    return cascade_flashed, flash_count, original_array

In [48]:
import cv2
import pandas as pd
import numpy as np

data = pd.read_csv('test_data.csv', header=None)
numbers = [str(no[0]) for no in data.values.tolist()]
original_array = []
for idx in range(len(numbers)):
    original_array.append([int(i) for i in numbers[idx]])
original_array = np.array(original_array)
original_array = cv2.copyMakeBorder(original_array, 1,1,1,1,cv2.BORDER_CONSTANT, value=[11])

flash_count = []
original_array = np.array(original_array) + 1
# coord = np.where(original_array == 10)
# while coord[0].size > 0:
#     coord, original_array = oct_flash(original_array, coord)


In [53]:
coord = np.where(original_array == 10)
len(coord[0])

8

In [50]:
original_array

array([[12, 12, 12, 12, 12, 12, 12],
       [12,  2,  2,  2,  2,  2, 12],
       [12,  2, 10, 10, 10,  2, 12],
       [12,  2, 10,  2, 10,  2, 12],
       [12,  2, 10, 10, 10,  2, 12],
       [12,  2,  2,  2,  2,  2, 12],
       [12, 12, 12, 12, 12, 12, 12]], dtype=int32)

In [46]:
original_array = np.array(original_array) + 1
coord = np.where(original_array == 10)
while coord[0].size > 0:
    coord, original_array = oct_flash(original_array, coord)

original_array

array([[21, 22, 23, 23, 23, 22, 21],
       [22,  3,  3,  3,  3,  3, 22],
       [23,  3,  3,  3,  3,  3, 23],
       [23,  3,  3,  3,  3,  3, 23],
       [23,  3,  3,  3,  3,  3, 23],
       [22,  3,  3,  3,  3,  3, 22],
       [21, 22, 23, 23, 23, 22, 21]], dtype=int32)